In [ ]:
# Run servers:
# central_server.py
# AS_server_twin.py
# Gatan_sever_twin.py
# like this from the main folder: python -m asyncroscopy.servers.protocols.central_server

In [ ]:
import sys
sys.path.insert(0, '../')
from asyncroscopy.clients.notebook_client import NotebookClient
import matplotlib.pyplot as plt

In [ ]:
# Connect the Client to the central (async) server
tem = NotebookClient.connect(host='localhost',port=9000)

# Tell the central server address of all connected instruments
routing_table= {"AS": ("localhost", 9001),
                "Gatan": ("localhost", 9002),
                "Ceos": ("localhost", 9003),
                "Preacquired_AS": ("localhost", 9004)}
tem.send_command('Central',"set_routing_table", routing_table)

In [ ]:
# connect to the AutoScript computer and initialize microscope
tem.send_command('AS',command='connect_AS',args={'host':'localhost','port':9001})


In [ ]:
stage = tem.send_command('AS',command='get_stage')
stage

In [ ]:
tem.send_command('AS',command='get_status') # if no args provided, uses {}

### Getting an image takes 5 seconds:

In [ ]:
image_args = {'scanning_detector':'HAADF',
                'size':512,
                'dwell_time':10e-6}

img = tem.send_command('AS','get_scanned_image', image_args)

plt.imshow(img, cmap="gray")
plt.title("Simulated STEM Image")
plt.show()


### Getting a spectrum takes 3 seconds:

In [ ]:
spec_args = {'size':512}

spec = tem.send_command('Gatan','get_spectrum', spec_args)
plt.plot(spec)
plt.title("Simulated Spectrum")
plt.show()

### Why not both at once?

In [ ]:
image, spec = tem.send_parallel_commands([('AS','get_scanned_image', image_args),
                                            ('Gatan','get_spectrum', spec_args)])

fig, ax = plt.subplots(1,2, figsize = (10,5))
ax[0].imshow(image, cmap="gray")
ax[1].plot(spec)
